# Imports

In [14]:
import os
import pandas as pd

pd.set_option("display.max_columns", None)

## Functions

In [15]:
def df_add_date_info(df):

    df_c = df.copy()

    df_c["Timestamp"] = df_c["Date"].astype(int)
    df_c["Week"] = df_c["Date"].dt.isocalendar().week
    df_c["Month"] = df_c["Date"].dt.month

    return df_c


def df_adjust_odds(df):

    df_c = df.copy()

    for i, row in df_c.iterrows():
        df_c.loc[i, "OddsAdjH"] = row["OddsH"] - 1 if row["H"] else -1
        df_c.loc[i, "OddsAdjA"] = row["OddsA"] - 1 if row["A"] else -1
        df_c.loc[i, "OddsAdjOnlyH"] = row["OddsH"] - 1 if row["H"] else 0
        df_c.loc[i, "OddsAdjOnlyA"] = row["OddsA"] - 1 if row["A"] else 0
        df_c.loc[i, "OddsW"] = row["OddsH"] if row["H"] else row["OddsA"] 

    df_c["OddsAdjW"] = df_c["OddsW"] - 1

    return df_c


def report_odds(df, groupby, season_filt=None):

    df_filt = df[df["Season"] > season_filt] if season_filt is not None else df.copy()

    c_m = ["H", "A", "OddsH", "OddsA", "OddsW"]
    c_s = ["OddsAdjH", "OddsAdjA", "OddsAdjOnlyH", "OddsAdjOnlyA", "OddsAdjW"]
    agg = {cc: "mean" for cc in c_m}
    agg.update({cc: "sum" for cc in c_s})
    agg["Timestamp"] = "max"
    agg["HID"] = "size"

    df_group = df_filt.groupby(groupby).agg(agg).sort_values(by="Timestamp")
    df_group["Date"] = pd.to_datetime(df_group["Timestamp"])
    df_group["OddsImProbH"] = 1 / df_group["OddsH"]
    df_group["OddsImProbA"] = 1 / df_group["OddsA"]
    df_group["OddsImProbW"] = 1 / df_group["OddsW"]
    df_group["G"] = df_group["HID"]

    c_format = {"H": "{:,.2%}".format, "A": "{:,.2%}".format, "OddsImProbH": "{:,.2%}".format, "OddsImProbA": "{:,.2%}".format, "OddsImProbW": "{:,.2%}".format}
    c_order = ["G", "H", "A", "OddsImProbH", "OddsImProbA", "OddsImProbW", "OddsAdjH", "OddsAdjA", "OddsAdjW", "OddsAdjOnlyH", "OddsAdjOnlyA"]

    vm = {}
    for c in ["OddsAdjH", "OddsAdjA", "OddsAdjOnlyH", "OddsAdjOnlyA", "OddsAdjW"]:
        vm[c] = max(abs(df_group[c].max()), abs(df_group[c].min()))

    display(df_group[c_order] \
        .style \
        .background_gradient(cmap="RdYlGn", subset=["OddsAdjH"], vmin=-vm["OddsAdjH"], vmax=vm["OddsAdjH"]) \
        .background_gradient(cmap="RdYlGn", subset=["OddsAdjA"], vmin=-vm["OddsAdjA"], vmax=vm["OddsAdjA"]) \
        .background_gradient(cmap="RdYlGn", subset=["OddsAdjW"], vmin=-vm["OddsAdjW"], vmax=vm["OddsAdjW"]) \
        .background_gradient(cmap="RdYlGn", subset=["OddsAdjOnlyH"], vmin=-vm["OddsAdjOnlyH"], vmax=vm["OddsAdjOnlyH"]) \
        .background_gradient(cmap="RdYlGn", subset=["OddsAdjOnlyA"], vmin=-vm["OddsAdjOnlyA"], vmax=vm["OddsAdjOnlyA"]) \
        .format(c_format))

# Games Data

In [ ]:
data_path = os.path.join("..", "data", "games.csv")
df = pd.read_csv(data_path, parse_dates=["Date", "Open"], date_format="%Y-%m-%d", index_col=0)

df = df[df["N"] == 0].drop(columns="N")

df = df_add_date_info(df)
df = df_adjust_odds(df)

display(df.head(), df.shape)

,Season,Date,HID,AID,POFF,Open,OddsH,OddsA,H,A,HSC,ASC,HFGM,AFGM,HFGA,AFGA,HFG3M,AFG3M,HFG3A,AFG3A,HFTM,AFTM,HFTA,AFTA,HORB,AORB,HDRB,ADRB,HRB,ARB,HAST,AAST,HSTL,ASTL,HBLK,ABLK,HTOV,ATOV,HPF,APF,Timestamp,Week,Month,OddsAdjH,OddsAdjA,OddsAdjOnlyH,OddsAdjOnlyA,OddsW,OddsAdjW
0,1,1975-11-07,13,28,0,1975-11-06,1.196537,4.833232,1,0,108,104,43.0,39.0,84.0,83.0,2.0,0.0,3.0,3.0,20.0,26.0,30.0,35.0,18.0,14.0,30.0,21.0,48.0,35.0,26.0,26.0,7.0,14.0,5.0,4.0,25.0,14.0,27.0,26.0,184550400000000000,45,11,0.196537,-1.0,0.196537,0.0,1.196537,0.196537
1,1,1975-11-07,20,12,0,1975-11-06,1.394620,2.957488,1,0,103,77,32.0,28.0,76.0,90.0,2.0,0.0,3.0,4.0,37.0,21.0,47.0,30.0,16.0,15.0,44.0,29.0,60.0,44.0,14.0,21.0,7.0,5.0,5.0,2.0,22.0,21.0,31.0,33.0,184550400000000000,45,11,0.394620,-1.0,0.394620,0.0,1.394620,0.394620
2,1,1975-11-07,43,8,0,1975-11-06,1.502570,2.536599,1,0,127,123,49.0,49.0,92.0,99.0,0.0,1.0,0.0,6.0,29.0,24.0,36.0,31.0,14.0,18.0,36.0,27.0,50.0,45.0,29.0,26.0,9.0,15.0,10.0,3.0,23.0,18.0,19.0,31.0,184550400000000000,45,11,0.502570,-1.0,0.502570,0.0,1.502570,0.502570
3,1,1975-11-07,33,4,0,1975-11-06,1.275743,3.778120,1,0,134,104,55.0,43.0,102.0,93.0,1.0,2.0,2.0,6.0,23.0,16.0,26.0,24.0,19.0,16.0,35.0,18.0,54.0,34.0,36.0,27.0,9.0,7.0,7.0,4.0,13.0,17.0,18.0,22.0,184550400000000000,45,11,0.275743,-1.0,0.275743,0.0,1.275743,0.275743
4,1,1975-11-07,41,34,0,1975-11-06,1.112895,7.379109,1,0,120,102,47.0,37.0,84.0,85.0,1.0,0.0,4.0,1.0,25.0,28.0,25.0,36.0,14.0,20.0,30.0,18.0,44.0,38.0,31.0,18.0,5.0,10.0,6.0,6.0,18.0,22.0,27.0,25.0,184550400000000000,45,11,0.112895,-1.0,0.112895,0.0,1.112895,0.112895


(28977, 49)

In [23]:
print("Reported Odds - Grouped by Season")
report_odds(df, ["Season"])
print("Reported Odds - Grouped by Season and Month")
report_odds(df, ["Season", "Month"], season_filt=20)
print("Reported Odds - Grouped by Season and Week")
report_odds(df, ["Season", "Week"], season_filt=20)

Reported Odds - Grouped by Season


,G,H,A,OddsImProbH,OddsImProbA,OddsImProbW,OddsAdjH,OddsAdjA,OddsAdjW,OddsAdjOnlyH,OddsAdjOnlyA
Season,,,,,,,,,,,
1,1014,66.86%,33.14%,58.72%,31.29%,55.84%,37.189754,-249.237287,801.952467,373.189754,428.762713
2,1023,68.13%,31.87%,58.50%,30.82%,56.50%,52.451465,-287.921755,787.529710,378.451465,409.078245
3,1087,66.97%,33.03%,55.43%,28.92%,55.70%,52.013771,-274.476198,864.537573,411.013771,453.523802
4,1179,65.06%,34.94%,56.11%,29.25%,55.87%,-6.043861,-241.730392,931.225747,405.956139,525.269608
5,1173,65.90%,34.10%,55.79%,29.09%,54.96%,33.915688,-245.652384,961.263304,433.915688,527.347616
6,1180,63.64%,36.36%,57.47%,30.20%,54.46%,-37.629271,-155.637443,986.733286,391.370729,595.362557
7,1181,61.39%,38.61%,56.43%,29.35%,52.84%,-64.113525,-62.820960,1054.065516,391.886475,662.179040
8,1184,61.82%,38.18%,56.09%,28.94%,54.50%,-83.044385,-112.352379,988.603235,368.955615,619.647621
9,1178,59.42%,40.58%,56.77%,29.48%,51.62%,-90.562793,16.665729,1104.102936,387.437207,716.665729


Reported Odds - Grouped by Season and Month


Reported Odds - Grouped by Season and Week


In [24]:
print("Reported Odds - Grouped by HID")
report_odds(df, ["HID"])
print("Reported Odds - Grouped by HID with Season > 20")
report_odds(df, ["HID"], season_filt=20)
print("Reported Odds - Grouped by HID and Season")
report_odds(df, ["Season", "HID"], season_filt=22)

Reported Odds - Grouped by HID


,G,H,A,OddsImProbH,OddsImProbA,OddsImProbW,OddsAdjH,OddsAdjA,OddsAdjW,OddsAdjOnlyH,OddsAdjOnlyA
HID,,,,,,,,,,,
3,605,42.15%,57.85%,40.53%,46.25%,53.86%,-110.776085,24.156542,518.380457,239.223915,279.156542
39,343,60.06%,39.94%,59.30%,33.27%,53.32%,-27.418561,-15.328587,300.252851,109.581439,190.671413
42,1003,58.82%,41.18%,52.36%,35.51%,54.89%,-21.650223,-157.189762,824.160016,391.349777,432.810238
35,1013,53.80%,46.20%,51.75%,39.28%,52.38%,-80.736323,-11.383456,920.880220,387.263677,533.616544
13,1017,54.47%,45.53%,55.27%,35.95%,51.79%,-129.002431,58.841374,946.838943,333.997569,612.841374
15,1092,68.41%,31.59%,65.88%,25.13%,53.35%,-48.558795,-88.412316,955.028888,296.441205,658.587684
18,1048,62.60%,37.40%,59.12%,30.96%,52.29%,-19.220527,-72.701828,956.077645,372.779473,583.298172
28,1045,66.51%,33.49%,64.18%,29.63%,53.57%,-24.859785,-114.261909,905.878306,325.140215,580.738091
8,987,52.99%,47.01%,50.40%,39.27%,53.41%,-95.289782,-30.781092,860.929126,368.710218,492.218908


Reported Odds - Grouped by HID with Season > 20


,G,H,A,OddsImProbH,OddsImProbA,OddsImProbW,OddsAdjH,OddsAdjA,OddsAdjW,OddsAdjOnlyH,OddsAdjOnlyA
HID,,,,,,,,,,,
3,164,40.85%,59.15%,44.82%,47.50%,51.02%,-30.168860,23.602982,157.434122,66.831140,90.602982
39,173,65.32%,34.68%,63.07%,29.88%,52.76%,-4.738290,-13.374599,154.887112,55.261710,99.625401
42,164,45.73%,54.27%,46.14%,46.12%,53.14%,-24.457219,5.087954,144.630735,64.542781,80.087954
35,170,44.71%,55.29%,47.87%,44.62%,51.79%,-37.123937,25.376522,158.252586,56.876063,101.376522
13,170,47.65%,52.35%,56.33%,39.96%,53.37%,-42.179291,20.697649,148.518357,46.820709,101.697649
15,183,60.11%,39.89%,64.16%,26.27%,48.57%,-30.006008,40.765974,193.759966,42.993992,150.765974
18,164,43.90%,56.10%,47.44%,48.73%,50.41%,-19.129781,16.462007,161.332225,72.870219,88.462007
28,164,55.49%,44.51%,54.33%,41.13%,52.81%,-7.719209,-9.734941,146.545849,65.280791,81.265059
8,169,59.17%,40.83%,55.38%,35.92%,50.24%,-3.176545,1.536156,167.359611,65.823455,101.536156


Reported Odds - Grouped by HID and Season
